### Setup

In [1]:
import sklearn as sk
import numpy as np
import pandas as pd
import os

## Import Data

In [2]:
dataFile = os.path.join(
    os.environ["PWD"],
    "data",
    "smallPrepared.csv"
)

df = pd.read_csv(dataFile, header=0, index_col=0)
print(len(df), " rows")
print(len(df.columns), " cols")
df.head()

7939  rows
5  cols


,dropperc,mins,consecmonths,income,churn_Y
custid,,,,,
844336,0.016364,550,28,89.2,0
146041,0.018349,545,33,54.2,0
847745,0.018519,378,41,55.3,0
285565,0.014493,552,32,66.8,0
754611,0.012132,577,4,87.2,0


### Begin Model Building

In [3]:
columns = list(df.columns)
columns.remove('churn_Y')
y = df["churn_Y"].values
X = df[columns].values

In [4]:
from sklearn.ensemble import AdaBoostClassifier
ad1 = AdaBoostClassifier(learning_rate=1)
ad1 = ad1.fit(X, y)
ad1prb = ad1.predict_proba(X)

In [5]:
from sklearn.ensemble import AdaBoostClassifier
ad2 = AdaBoostClassifier(learning_rate=0.5)
ad2 = ad2.fit(X, y)
ad2prb = ad2.predict_proba(X)

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
gb1 = GradientBoostingClassifier(loss = 'exponential', max_depth=3)
gb1 = gb1.fit(X, y)
gb1prb = gb1.predict_proba(X)

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb2 = GradientBoostingClassifier(loss = 'exponential', max_depth=10)
gb2 = gb2.fit(X, y)
gb2prb = gb2.predict_proba(X)

In [8]:
from sklearn.ensemble import RandomForestClassifier 
rf1 = RandomForestClassifier(max_depth=None)
rf1 = rf1.fit(X, y)
rf1prb = rf1.predict_proba(X)
rf1pclass = rf1.predict(X)

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [9]:
from sklearn.ensemble import RandomForestClassifier 
rf2 = RandomForestClassifier(max_depth = 5)
rf2 = rf2.fit(X, y)
rf2prb = rf2.predict_proba(X)
rf2pclass = rf2.predict(X)

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [10]:
from sklearn.ensemble import RandomForestClassifier 
rf3 = RandomForestClassifier(max_depth = 30)
rf3 = rf3.fit(X, y)
rf3prb = rf3.predict_proba(X)
rf3pclass = rf3.predict(X)

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [11]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg = logreg.fit(X, y)
logregprb = logreg.predict_proba(X)
logregpclass = logreg.predict(X)

/usr/local/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [12]:
print(logreg.intercept_)
print(logreg.coef_)

[-2.61171836]
[[ 0.40323489 -0.0090457   0.09232167  0.01609967]]


### Build a Cross Validation Function

In [13]:
from sklearn import cross_validation

# takes a list of models, the input np.array, the target np.array, the type of score to be used with cv, and k
# each element in the list of models should have two items: the model object and the name you want to use for that 
# model object
# returns a dataframe with the names you entered and the mean of the cv scores across all k folds

def cv_fun(models, inputs, target, score, k):
    i = 0
    for m in models:
        scores = cross_validation.cross_val_score(models[i][0], inputs, target, scoring=score, cv=k)
    
        if i==0:
            list1 = list()
            list2 = list()
            
        list1.append(round(scores.mean(),3))
        list2.append(models[i][1])

        i=i+1

    return pd.DataFrame(list1, index=list2, columns=[score])

ImportError: cannot import name 'cross_validation'

### Cross Validation on the Training Data

In [ ]:
# use the cv function found up under the Setup section
# enter a list with each entry holding the model object followed by a text name you want to give the model

input_models = [[ad1, 'ad1']]
input_models.append([ad2, 'ad2'])
input_models.append([gb1, 'gb1'])
input_models.append([gb2, 'gb2'])
input_models.append([rf1, 'rf1'])
input_models.append([rf2, 'rf2'])
input_models.append([rf3, 'rf3'])
input_models.append([logreg, 'logreg'])

cv_auc = cv_fun(input_models, X, y, 'roc_auc', 5)
cv_acc = cv_fun(input_models, X, y, 'accuracy', 5)
cv_results = cv_auc.join(cv_acc)
cv_results.sort_values(by='roc_auc', ascending=False, inplace=True)

In [ ]:
#manually pick best, would be nice to automate this part
cv_results

### Save Model to File

In [ ]:
import pickle 
best = gb1
pickle.dump(best, open('/mnt/results/smallModel.pkl', 'wb')) # w = open for writing, r = open for reading, b = binary

In [ ]:

#make confusion matrix plot
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot
 
    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix
 
    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']
 
    title:        the text to display at the top of the matrix
 
    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues
 
    normalize:    If False, plot the raw numbers
                  If True, plot the proportions
 
    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph
 
    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
 
    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools
 
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
 
    if cmap is None:
        cmap = plt.get_cmap('Blues')
 
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
 
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)
 
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
 
 
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
 
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.gcf().subplots_adjust(bottom=0.25)
    plt.savefig('/mnt/results/ConfMatx_Best.png', format='png')
    plt.show()
    plt.gcf().clear()
    


In [ ]:
from sklearn import metrics

plot_confusion_matrix(cm           = metrics.confusion_matrix(y, best.predict(X)), 
                      normalize    = False,
                      target_names = ['no churn', 'churn'],
                      title        = "Confusion Matrix for Best Model")
                      

In [ ]:
df2 = df
df2["prob"] = best.predict_proba(X)[:,1]

In [ ]:
df2.head()

In [ ]:
df2.to_csv('/mnt/data/smallModelOut.csv')